In [ ]:
import random
import numpy as np
import math

# Task 1

In [ ]:
class Individual:
    def __init__(self, n, bit_values):
        self.n = n
        self.bit_values = bit_values
    
    def mutate(self):
        mutated_bit_values = self.bit_values.copy()
        k = np.random.binomial(self.n, 1-1/self.n)
        indices = random.sample(range(self.n), k)
        for i in indices:
            mutated_bit_values[i] = 1 - mutated_bit_values[i]
        return Individual(self.n, mutated_bit_values)
    
    
# def flip_bits_randomly(bits, p):   (geometric distribution version)
#     n = len(bits)
#     i = 0
#     while i < n:
#         # geometric distribution: number of trials until success
#         # geometric(p) can be sampled via inverse CDF
#         skip = int(math.log(1 - random.random()) / math.log(1 - p))

#         i += skip
#         if i >= n:
#             break

#         bits[i] ^= 1   # flip the bit
#         i += 1

In [ ]:
class Frequency_Vector :
    def __init__(self, n, values=None): 
        self.n = n
        if values is None:
            self.values = [1/n]*n
        else :
            for v in values:
                v = max(min(v,1-1/n),1/n)
                self.values.append(v)

    
    def update(self, i, p_i):
        self.values[i] = max(min(p_i,1-1/self.n),1/self.n)
        
    # def get(self, i):
    #     return self.values[i]
    
    def generate_individual(self):
        bit_values = []
        for i in range(self.n):
            r = random.random()
            if r < self.values[i]:
                bit_values.append(1)
            else:
                bit_values.append(0)
        return Individual(self.n, bit_values)
    
    

# Task 2

In [ ]:
def sig(p, H, eps, n):
    m = H.size()
    limit = m*p+eps*max(np.sqrt(m*p*np.log(n)), np.log(n))
    if limit <= H.count_ones() and p in [1/n,1/2]:
        return 1 # UP
    elif limit <= H.count_zeros() and p in [1-1/n,1/2]:
        return -1 # DOWN
    else:
        return 0 # STAY
    

In [ ]:
class Simplified_History :
    def __init__(self, size=0, zeros=0, ones=0):
        self.size = size
        self.zeros = zeros
        self.ones = ones
    
    def add(self, ind):
        self.size+=1
        if ind == 1:
            self.ones+=1
        else:
            self.zeros+=1
    
    def size(self):
        return self.size
    
    def count_ones(self):
        return self.ones
    
    def count_zeros(self):
        return self.zeros

In [ ]:
class Original_History :
    def __init__(self, size, ones, next=None):
        self.size = size
        self.ones = ones        
        self.next = next
        
    def size(self):
        return len(self.size)
    
    def count_ones(self):
        return sum(self.ones)
    
    def count_zeros(self):
        return len(self.size) - sum(self.ones)
    
    def add(self, ind, n):
        if self.size < n:
            self.size +=1
            if ind == 1:
                self.ones +=1
        else:
            self = Original_History(1, ind, self)
            consolidate(self)
    
    
def consolidate(L):
    curr = L
    if curr is None:
        next = curr.next
        alreadySeenDouble = False
        while next is not None:
            if curr.size == next.size:
                if alreadySeenDouble:
                    m = Original_History(curr.size + next.size, curr.ones + next.ones, curr.next)
                    curr = m
                    curr.next = next.next
                    next = curr.next
                    alreadySeenDouble = False
                else:
                    alreadySeenDouble = True
                    curr = next
            else:
                curr = next     

The maximum number of merges is $O(log(m))$ where $m$ is the number of bits added so far, this happens when each block size doubles, for example: 

# Task 3

In [ ]:
def sig_cga(f, termination_condition, n):
    t = 0
    x_1 = None # have to initialize x_1 for the termination condition
    p = Frequency_Vector(n, [0.5]*n)
    H = [Original_History(0, 0, None) for _ in range(n+1)]
    while not termination_condition(t, x_1):  # have to depend on an individual ?
        x_1 = p.generate_individual()
        x_2 = p.generate_individual()
        if f(x_1) < f(x_2):
            x_1, x_2 = x_2, x_1
        for i in range(n):
            H[i].add(x_1[i], n)
            curr = H[i]
            h = Original_History(0, 0, None)
            while curr is not None:
                h = Original_History(h.size + curr.size, h.ones + curr.ones, None)
                s = sig(p.values[i], h, 1/n, n)
                if s == 1:
                    p.update(i, p.values[i]+1/n)
                elif s == -1:
                    p.update(i, p.values[i]-1/n)
                curr = curr.next
                if s != 0:
                    H[i] = Original_History(0, 0, None)
                    break
        t+=1

# Task 4

In [ ]:
def cga(K, n, f, termination_criterion):
    t = 0
    x_1 = None
    p = Frequency_Vector(n, [0.5]*n)
    while not termination_criterion(t, x_1):
        x_1 = p.generate_individual()
        x_2 = p.generate_individual()
        if f(x_1) < f(x_2):
            x_1, x_2 = x_2, x_1
        for i in range(n):
            p.update(i, p.values[i] + 1/K * (x_1.bit_values[i] - x_2.bit_values[i]))
        t+=1

def EA(f, termination_criterion, n):
    t = 0
    x = Individual(n, [random.randint(0, 1) for _ in range(n)])
    while not termination_criterion(t, x):
        y = x.mutate()
        if f(y) > f(x):
            x = y
        t+=1

# Task 5

In [ ]:
def OneMax(ind):
    return sum(ind.bit_values)

def LeadingOnes(ind):
    count = 0
    for bit in ind.bit_values:
        if bit == 1:
            count += 1
        else:
            break
    return count

def Jump(ind, k):
    n = ind.n
    ones = sum(ind.bit_values)
    if ones == n or ones <= n-k:
        return k + ones
    else:
        return n - ones

eval = 100
def termination_criterion(t, x):
    return t >= eval or x.bit_values == [1]*x.n

# Task 6

In [ ]:
random.seed(42)